In [1]:
import os
import json
import tensorflow as tf
import numpy as np
from pycocotools.cocoeval import COCOeval

from detection.datasets import coco, data_generator
from detection.models.detectors import faster_rcnn
from tensorflow.python.keras.layers.normalization import BatchNormalization
from tqdm.notebook import tqdm

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
img_mean = (123.675, 116.28, 103.53)
# img_std = (58.395, 57.12, 57.375)
img_std = (1., 1., 1.)
batch_size = 8
val_dataset = coco.CocoDataSet('/workspace/shared_workspace/data/coco/', 'val',
                               flip_ratio=0,
                               pad_mode='fixed',
                               mean=img_mean,
                               std=img_std,
                               scale=(800, 1216))

val_generator = data_generator.DataGenerator(val_dataset, shuffle=True)
val_tf_dataset = tf.data.Dataset.from_generator(
    val_generator, (tf.float32, tf.float32, tf.float32, tf.int32, tf.int32))

val_tf_dataset = val_tf_dataset.prefetch(16) #.shuffle(100).shard(hvd.size(), hvd.rank())
'''val_tf_dataset = val_tf_dataset.padded_batch(
                            batch_size,
                            padded_shapes=(
                            tf.TensorShape([None, None, 3]), # image padded to largest in batch
                            tf.TensorShape([11]),            # image meta - no padding
                            tf.TensorShape([None, 4]),       # bounding boxes, padded to longest
                            tf.TensorShape([None]),           # labels, padded to longest
                            tf.TensorShape([None])),
                            padding_values=(0.0, 0.0, 0.0, -1, -1))'''

# flip channel order
def flip_channels(img, img_meta, bbox, label, labels):
    img = tf.reverse(img, axis=[-1])
    return img, img_meta, bbox, label

val_tf_dataset = val_tf_dataset.filter(lambda w, x, y, z, a: tf.equal(tf.shape(w)[0], batch_size))
val_tf_dataset = val_tf_dataset.map(flip_channels, num_parallel_calls=16)
val_tf_dataset = iter(val_tf_dataset.repeat())


loading annotations into memory...
Done (t=0.54s)
creating index...
index created!


In [4]:
val_tf_dataset = tf.data.Dataset.from_generator(
    val_generator, (tf.float32, tf.float32, tf.float32, tf.int32, tf.int32))
val_tf_dataset = val_tf_dataset.prefetch(16)
val_tf_dataset = val_tf_dataset.padded_batch(
                            batch_size,
                            padded_shapes=(
                            tf.TensorShape([None, None, 3]), # image padded to largest in batch
                            tf.TensorShape([11]),            # image meta - no padding
                            tf.TensorShape([None, 4]),       # bounding boxes, padded to longest
                            tf.TensorShape([None]),           # labels, padded to longest
                            tf.TensorShape([None])),
                            padding_values=(0.0, 0.0, 0.0, -1, -1))
def flip_channels(img, img_meta, bbox, label, labels):
    img = tf.reverse(img, axis=[-1])
    return img, img_meta, bbox, label, labels

val_tf_dataset = val_tf_dataset.filter(lambda w, x, y, z, a: tf.equal(tf.shape(w)[0], batch_size))
val_tf_dataset = val_tf_dataset.map(flip_channels, num_parallel_calls=16)
val_tf_dataset = iter(val_tf_dataset.repeat())

In [5]:
@tf.function(experimental_relax_shapes=True)
def evaluate_tf_function(imgs, img_metas):
    c2, c3, c4, c5 = model.backbone(imgs, training=False)
    p2, p3, p4, p5, p6 = model.neck((c2, c3, c4, c5), training=False)
    rpn_class_logits, rpn_probs, rpn_deltas = model.rpn_head((p2, p3, p4, p5, p6), training=False)
    proposals = model.rpn_head.get_proposals(rpn_probs, rpn_deltas, img_metas)
    pooled_regions = model.roi_align((proposals, (p2, p3, p4, p5), img_metas), training=False)
    cnn_class_logits, rcnn_probs, rcnn_deltas = \
                model.bbox_head(pooled_regions, training=False)
    detections_list = model.bbox_head.get_bboxes(
                    rcnn_probs, rcnn_deltas, proposals, img_metas)
    return detections_list

In [6]:
model = faster_rcnn.FasterRCNN(
    num_classes=len(val_dataset.get_categories()), batch_size=batch_size)

imgs, img_metas, bboxes, labels, ids = next(val_tf_dataset)

_ = model((imgs, img_metas, bboxes, labels))

In [7]:
model.layers[0].trainable=False

for layer in model.layers[0].layers[0].layers[142:]:
    if type(layer)!=BatchNormalization:
        layer.trainable=True

In [8]:
model.load_weights( \
'/workspace/shared_workspace/rcnn_keras_resnet_50_stage_2.h5')

In [122]:
detections_list = []
img_ids = []
metas = []
for i in tqdm(range(100)):
    imgs, img_metas, bboxes, labels, ids = next(val_tf_dataset)
    detections_list.append(evaluate_tf_function(imgs, img_metas))
    img_ids.append(ids)
    metas.append(img_metas)

In [123]:
def get_detections_single(detection, img_id, meta):
    detection = tf.gather_nd(detection, tf.where(tf.reduce_mean(detection, axis=1)))
    boxes = detection[...,:4].numpy()
    labels = detection[...,4].numpy()
    probs = detection[...,5].numpy()
    boxes = boxes[...,[1, 0, 3, 2]]/meta[-2].numpy()
    boxes[..., 2] = boxes[..., 2] - boxes[..., 0] 
    boxes[..., 3] = boxes[..., 3] - boxes[..., 1] 
    json_list = []
    for box, label, score in zip(boxes, labels, probs):
        json_list.append({"image_id": int(img_id.numpy()[0]), "category_id": int(label), "bbox": box.tolist(), "score": float(score)})
    return json_list

def get_detection_batch(detections_batch, img_id_batch, meta_batch):
    json_list = []
    for detection, img_id, meta in zip(detections_batch, img_id_batch, meta_batch):
        json_list.extend(get_detections_single(detection, img_id, meta))
    return json_list

def get_detections(detections_list, img_ids, metas):
    json_list = []
    for detections_batch, img_id_batch, meta_batch in zip(detections_list, img_ids, metas):
        json_list.extend(get_detection_batch(detections_batch, img_id_batch, meta_batch))
    return json_list
    

In [124]:
json_list = get_detections(detections_list, img_ids, metas)

In [125]:
import json
with open('eval_pred.json', 'w') as outfile:
    outfile.write(json.dumps(json_list))

In [126]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [127]:
cocoDt = val_dataset.coco.loadRes('eval_pred.json')
cocoEval = COCOeval(val_dataset.coco,cocoDt,'bbox')

Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!


In [128]:
cocoEval.params.imgIds = tf.reshape(tf.concat(img_ids, axis=0), [-1]).numpy().tolist()

In [129]:
cocoEval.evaluate()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.75s).


In [130]:
cocoEval.accumulate()

Accumulating evaluation results...
DONE (t=0.44s).


In [131]:
cocoEval.summarize()

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.005
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.015
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.003
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.005
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.007
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.005
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.009
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.010
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.006
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.010
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.014


In [1]:
import pathlib

In [4]:
pathlib.Path('eval_pred.json').unlink()

In [ ]:
import os
import json
import tensorflow as tf
import numpy as np
from pycocotools.cocoeval import COCOeval

from detection.datasets import coco, data_generator
from detection.models.detectors import faster_rcnn
from tensorflow.python.keras.layers.normalization import BatchNormalization
from tqdm.notebook import tqdm

class Evaluator(object):
    
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.val_tf_dataset = self.get_eval_data()
        
    def get_eval_data(self):
        mg_mean = (123.675, 116.28, 103.53)
        # img_std = (58.395, 57.12, 57.375)
        img_std = (1., 1., 1.)
        batch_size = 8
        val_dataset = coco.CocoDataSet('/workspace/shared_workspace/data/coco/', 'val',
                                       flip_ratio=0,
                                       pad_mode='fixed',
                                       mean=img_mean,
                                       std=img_std,
                                       scale=(800, 1216))

        val_generator = data_generator.DataGenerator(val_dataset, shuffle=True)        
        val_tf_dataset = tf.data.Dataset.from_generator(
            val_generator, (tf.float32, tf.float32, tf.float32, tf.int32, tf.int32))
        val_tf_dataset = val_tf_dataset.prefetch(16)
        val_tf_dataset = val_tf_dataset.padded_batch(
                                    batch_size,
                                    padded_shapes=(
                                    tf.TensorShape([None, None, 3]), # image padded to largest in batch
                                    tf.TensorShape([11]),            # image meta - no padding
                                    tf.TensorShape([None, 4]),       # bounding boxes, padded to longest
                                    tf.TensorShape([None]),           # labels, padded to longest
                                    tf.TensorShape([None])),
                                    padding_values=(0.0, 0.0, 0.0, -1, -1))
        def flip_channels(img, img_meta, bbox, label, labels):
            img = tf.reverse(img, axis=[-1])
            return img, img_meta, bbox, label, labels

        val_tf_dataset = val_tf_dataset.filter(lambda w, x, y, z, a: tf.equal(tf.shape(w)[0], batch_size))
        val_tf_dataset = val_tf_dataset.map(flip_channels, num_parallel_calls=16)
        val_tf_dataset = iter(val_tf_dataset.repeat())
        return val_tf_dataset
    
def get_detections_single(detection, img_id, meta):
    detection = tf.gather_nd(detection, tf.where(tf.reduce_mean(detection, axis=1)))
    boxes = detection[...,:4].numpy()
    labels = detection[...,4].numpy()
    probs = detection[...,5].numpy()
    boxes = boxes[...,[1, 0, 3, 2]]/meta[-2].numpy()
    boxes[..., 2] = boxes[..., 2] - boxes[..., 0] 
    boxes[..., 3] = boxes[..., 3] - boxes[..., 1] 
    json_list = []
    for box, label, score in zip(boxes, labels, probs):
        json_list.append({"image_id": int(img_id.numpy()[0]), "category_id": int(label), "bbox": box.tolist(), "score": float(score)})
    return json_list

def get_detection_batch(detections_batch, img_id_batch, meta_batch):
    json_list = []
    for detection, img_id, meta in zip(detections_batch, img_id_batch, meta_batch):
        json_list.extend(get_detections_single(detection, img_id, meta))
    return json_list

def get_detections(detections_list, img_ids, metas):
    json_list = []
    for detections_batch, img_id_batch, meta_batch in zip(detections_list, img_ids, metas):
        json_list.extend(get_detection_batch(detections_batch, img_id_batch, meta_batch))
    return json_list
    
    def evaluate(detections, img_ids, )